In [ ]:
pip install azure-eventhub

StatementMeta(, 1dcc5e94-8042-4391-8aec-daf748a63d80, 3, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 kB 12.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests, json, time
from statistics import mean
from azure.eventhub import EventHubProducerClient, EventData
from datetime import datetime, timezone, timedelta
import notebookutils
import sys
import uuid

# ==== FABRIC EVENTSTREAM CONNECTION DETAILS ====
CONNECTION_STRING ="Endpoint=sb://esehbn6l7tel4ue1dttd1o.servicebus.windows.net/;SharedAccessKeyName=key_64370351-29c8-47d3-83d4-5856ddfd5c18;SharedAccessKey=w67WIzx6m3kiB+JvNHTy3wA7xzohOUbzK+AEhDkINUg=;EntityPath=es_a5ec48a1-1306-4d21-a157-8e4d083b067e"
producer = EventHubProducerClient.from_connection_string(conn_str=CONNECTION_STRING)
IST = timezone(timedelta(hours=5, minutes=30))
# ==== TOMTOM TRAFFIC API CONFIG ====
API_KEY = "NhWEwanvL7p8Bd6hGqVc3lhVWVOcFp1P"
points = {
    "Connaught Place": (28.6139, 77.2090),
    "Noida": (28.5355, 77.3910),
    "West Delhi": (28.7041, 77.1025),
    "Gurugram": (28.4595, 77.0266),
    "New Delhi":(28.63,77.22)
}

# Get the current datetime object
now = datetime.now()
# Convert to Unix timestamp (float, seconds since epoch)
timestamp_seconds = now.timestamp()
dt_str = datetime.now(IST).isoformat()
dt_obj = datetime.fromisoformat(dt_str)
formatted_dt = dt_obj.strftime("%Y-%m-%d %H:%M")
# Convert to milliseconds and cast to an integer
timestamp_millis = int(timestamp_seconds * 1000)
print(timestamp_millis)
#run_id = notebookutils.getParameter("runId")
#print(f"Pipeline Run ID: {run_id}")
# ==== FETCH TRAFFIC DATA ====
events = []
for city, (lat, lon) in points.items():
    url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/relative0/10/json?point={lat},{lon}&key={API_KEY}"
    r = requests.get(url)
    data = r.json()["flowSegmentData"]

    current = data["currentSpeed"]
    free = data["freeFlowSpeed"]
    t_cur = data["currentTravelTime"]
    t_free = data["freeFlowTravelTime"]

    congestion_index = (1 - current / free) * 100
    delay = (t_cur - t_free) / 60

    event = {
        "City": city,
        "Latitude": lat,
        "Longitude": lon,
        "AverageSpeed_kmph": current,
        "CongestionIndex_percent": round(congestion_index, 2),
        "PeakHourDelay_mins": round(delay, 2),
        "dateTime": formatted_dt,        
        "timestamp_millis": timestamp_millis
    }
    events.append(event)
    
# ==== SEND TO FABRIC EVENTSTREAM ====
event_batch = producer.create_batch()
for e in events:
    event_batch.add(EventData(json.dumps(e)))

producer.send_batch(event_batch)
producer.close()
print(events)

print("✅ Sent traffic data to Fabric Eventstream successfully!")



StatementMeta(, 1dcc5e94-8042-4391-8aec-daf748a63d80, 4, Finished, Available, Finished)

1761924520986
[{'City': 'Connaught Place', 'Latitude': 28.6139, 'Longitude': 77.209, 'AverageSpeed_kmph': 23, 'CongestionIndex_percent': 4.17, 'PeakHourDelay_mins': 0.2, 'dateTime': '2025-10-31 20:58', 'timestamp_millis': 1761924520986}, {'City': 'Noida', 'Latitude': 28.5355, 'Longitude': 77.391, 'AverageSpeed_kmph': 27, 'CongestionIndex_percent': 12.9, 'PeakHourDelay_mins': 5.98, 'dateTime': '2025-10-31 20:58', 'timestamp_millis': 1761924520986}, {'City': 'West Delhi', 'Latitude': 28.7041, 'Longitude': 77.1025, 'AverageSpeed_kmph': 22, 'CongestionIndex_percent': 26.67, 'PeakHourDelay_mins': 4.27, 'dateTime': '2025-10-31 20:58', 'timestamp_millis': 1761924520986}, {'City': 'Gurugram', 'Latitude': 28.4595, 'Longitude': 77.0266, 'AverageSpeed_kmph': 44, 'CongestionIndex_percent': 10.2, 'PeakHourDelay_mins': 3.18, 'dateTime': '2025-10-31 20:58', 'timestamp_millis': 1761924520986}, {'City': 'New Delhi', 'Latitude': 28.63, 'Longitude': 77.22, 'AverageSpeed_kmph': 24, 'CongestionIndex_percen